In [15]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#import matplotlib.patches as mpatches
#from mycolorpy import colorlist as mcp

from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.metrics import f1_score
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm

#from PIL import Image
#import requests
#from transformers import CLIPModel, CLIPProcessor, AutoTokenizer
#from numba import njit, prange
import gc
from io import StringIO
#gc.collect()
plt.style.use('fivethirtyeight')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
def divariance_dict(arr_a, arr_b, c_type='dirrelation'):
    """c_type: {
    dvr: divariance    variance of swapped members of two distributions
    sdd: standard dideviation - root divariance, compare to standard deviation
    dirrelation: sigma product/divariance     compare to correlation, but measures deviation of magnitude vs direction. sigma product is the pearson denominator (sigma_a*sigma_b)
    divergence: 1- dirrelation   "gain";  normalized measure of difference. double this and it approximates KL divergence between two somewhat normal distributions

    
    calc_method: which method for calculating cross var... keeping for progression, but 'ms_sm' is unequivically better
    ms_sm: mean of squares - square of means.  no matrices, based on crossvar proof (analagous to variance proof)
    einsum: np.einsum version... still rectangular matrix calc, but faster than np_broadcast
    np_broadcast: numpy broadcasting.  faster than iterating, but still rectangular matrix calc
    """
    ret_dict = {}
    calc_method = 'ms_sm'     
    if calc_method == 'ms_sm':
        a_mean, b_mean = arr_a.mean(axis=0), arr_b.mean(axis=0)
        a_sqr_mean, b_sqr_mean = np.power(arr_a,2).mean(axis=0) , np.power(arr_b,2).mean(axis=0) 
        dvr = (a_sqr_mean+b_sqr_mean)/2 - a_mean*b_mean
        ret_dict['dvr']=dvr
    if calc_method == 'einsum':
        # the following is a 50% faster way to calc crossvar, but harder to understand... see the commented code below
        difference = (arr_a[:] - arr_b[:,None])
        ret_dict['dvr'] = np.einsum('ij...,ij...->...', difference, difference) /(len(arr_a)*len(arr_b)*2)
    if calc_method == 'np_broadcast':
        # the code below is more straight forward to understand... broadcast, take difference, square, take mean, div 2
        ret_dict['dvr'] = np.power((arr_a[:] - arr_b[:,None]),2).mean(axis=(0,1))/2
    
    ret_dict['sdd'] = np.sqrt(ret_dict['dvr'])
    ret_dict['dirrelation'] = (arr_a.std(axis=0)*arr_b.std(axis=0))/ ret_dict['dvr']  
    ret_dict['divergence'] = 1 - ret_dict['dirrelation'] #double this and it approximates KL divergence between two somewhat normal distributions
    return ret_dict[c_type]

In [17]:
def generate_distributions(mean1, mean2, var1, var2, corr, size=1000):
    """
    Generate two distributions with specified means, variances, and covariance.

    Parameters:
        mean1 (float): Mean of the first distribution.
        mean2 (float): Mean of the second distribution.
        var1 (float): Variance of the first distribution.
        var2 (float): Variance of the second distribution.
        #cov (float): Covariance between the two distributions.
        corr (float): correlation between the two distributions
        size (int): Number of samples to generate.

    Returns:
        x (np.ndarray): Samples from the first distribution.
        y (np.ndarray): Samples from the second distribution.
    """
    # calculate covar
    cov = corr*np.sqrt(var1)*np.sqrt(var2)
    
    # Create the covariance matrix
    cov_matrix = np.array([[var1, cov],
                           [cov, var2]])
    # Mean vector
    mean_vector = np.array([mean1, mean2])
    # Generate samples
    samples = np.random.multivariate_normal(mean_vector, cov_matrix, size)
    x, y = samples[:, 0], samples[:, 1]
    return x, y


In [18]:
def divariance(arr_a, arr_b):
    """dvr: divariance    variance of swapped members of two distributions
    I subtract one from denomanator to match covariance... only meaninful for small sample sizes"""
    a_mean, b_mean = arr_a.sum(axis=-1)/(arr_a.shape[-1]-1), arr_b.sum(axis=-1)/(arr_b.shape[-1]-1)#arr_a.mean(axis=-1), arr_b.mean(axis=-1)
    a_sqr_mean, b_sqr_mean = np.power(arr_a,2).sum(axis=-1)/(arr_a.shape[-1]-1) , np.power(arr_b,2).sum(axis=-1)/(arr_b.shape[-1]-1)#np.power(arr_a,2).mean(axis=-1) , np.power(arr_b,2).mean(axis=-1) 
    dvr = (a_sqr_mean+b_sqr_mean)/2 - a_mean*b_mean
    return dvr

def sdd(arr_a,arr_b):
    """sdd: standard dideviation - root divariance, compare to standard deviation """
    return np.sqrt(divariance(arr_a, arr_b))

def dirrelation(arr_a,arr_b):
    """dirrelation: sigma product/divariance     compare to correlation, but measures deviation of magnitude vs direction. sigma product is the pearson denominator (sigma_a*sigma_b)"""
    return (arr_a.std(axis=-1)*arr_b.std(axis=-1))/ divariance(arr_a, arr_b)

def dvr_gain(arr_a,arr_b):
    """gain: 1- dirrelation   "divergence";  normalized measure of difference. double this and it approximates KL divergence between two somewhat normal distributions"""
    return 1 - dirrelation(arr_a,arr_b)

- divariance
- dirrelation
- gain/divergence
- mean squared magnitude (MSM)
- magnitude similarity
- vector similarity
- frechet direlation distance (name is tough on this one - inception distance - what does that mean?)  portfolio gain?  no... reserve that idea for later.  really its the distance between two distributions across all features, while taking into account the interrelations of each feature (covar matrix)

In [19]:
# vector work
"""

"""
arr_a = np.random.random((1,100))
arr_b = np.random.random((1,100))
arr_a.shape

(1, 100)

In [20]:
vectors = [np.array([np.random.random((1,100))+1,np.random.random((1,100))]).squeeze(),np.array([(np.random.random((1,100))*1.5)+1,1.5*np.random.random((1,100))]).squeeze()]
v0 = vectors[0].copy()
v1 = vectors[1].copy()


In [21]:
def vnorm(arr_a):
    return np.sqrt((np.square(arr_a).sum(axis=-1)))

def vnorm_prep(v0,v1):
    sm0 = vnorm(v0) 
    sm1 = vnorm(v1) 
    if len(v0.shape) < 2:
        sm0 = (np.array([sm0]))
    if len(v1.shape) < 2:
        sm1 = (np.array([sm1]))
    return sm0,sm1
    
def msm(v0,v1):
    """mean squared magnitude: (|X|^2+|Y|^2)/2
    average squared magnitude of two (or more) vectors;  core operator for divariance;  compare to dot product """
    sm0,sm1 = np.square(vnorm_prep(v0,v1))
    
    return (sm0[:,np.newaxis]+sm1[np.newaxis,:])/2

def sigma_product(v0,v1):
    """product of standard deviations of two distributions.  Denomenator of Pearson Correlation"""
    sm0,sm1 = vnorm_prep(v0,v1)
      
    return sm0[:,np.newaxis]*sm1[np.newaxis,:]

def magnitude_similarity(v0,v1):
    """Measure of similarity of two vectors by magnitude, independent of direction
    std(v0)*std(v1) / mean squared magnitude
    """
    return sigma_product(v0,v1)/msm(v0,v1)

def vector_similarity(v0,v1):
    """normalized measure of how two vectors are similar in both direction and magnitude 
    cosine_similarity * magnitude similarity
    """
    return cosine_similarity(v0,v1) * magnitude_similarity(v0,v1)

In [22]:
#sigma_product(v0,v1)
magnitude_similarity(v0,v1)

array([[0.98428188, 0.8525859 ],
       [0.56974411, 0.92358325]])

In [24]:


def fid(act1, act2):
	# calculate mean and covariance statistics
    """frechet inception distance numpy implementation
    adapted from https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/ 
    I changed the rowvar to =True so that the covariance matrix is captured over the features instead of the samples.  
    Since FID is cumulative, you're getting a massive number. That is capturing the covariance between each of the samples,
    not each of the features (activations).Since we're trying to measure the relative relationship between each feature within 
    the collection (and then compare to the other collection), you have to do it feature-wise. 
    In that article cited I think they talk about 'walking the dog'... determining how well the leash length between 
    the first distribution (the walker) and the second (the dog) stays constant."""
    
	mu1, sigma1 = act1.mean(axis=1), cov(act1, rowvar=True)
	mu2, sigma2 = act2.mean(axis=1), cov(act2, rowvar=True)
	# calculate sum squared difference between means
	ssdiff = np.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

def vardist(act1,act2,return_features=False):
    """variational inception distance - modeled on frechet inception distance using divariance and shared covariance between two distributions of multi-feature data
    should have application whenever there are multiple samples over multiple features and two distinct categories... 
    that applies to tuning models on the same set of image/text data, classifying based on multiple features of the same model, clustering observations within a single data set/model.
    divariance(act1,act2).sum() - dot_covariance.sum()
    if return_features==True, return the difference by feature, otherwise return the sum of all features
    As the feature means, variances and correlations between the two distributions converge, this will converge to 0
    """
    if return_features:
        return divariance(act1,act2) - frechet_covariance(act1,act2)
    else:
        return divariance(act1,act2).sum() - frechet_covariance(act1,act2).sum()

def frechet_covariance(act1,act2):
    """feature wise shared covariance between two multi-dimensional distributions. Captures the degree to which there is covariance within each distribition 
    and how the covariance compares between the distributions.
    diagnoal of sqrt(Covar_matrix_a .dot covar_matrix_b) 
    find out what the paper called this term.  I could call it dotco for short.  dot implies two linked distributions... we'll try it.
    no reason to calculate the covmean intermediate step.  The trace will be a shortcut to the sum... could integrate it, but not necessary now I think
    As the feature correlation pattern between the two models converges, this value will converge to the product of the variances of the distributions... 
    or the product of the diagonals of the root internal covariance matrices
    """
    sigma1, sigma2 = cov(act1, rowvar=True), cov(act2, rowvar=True)
    return np.diag(sqrtm(sigma1.dot(sigma2)).real)

def frechet_dicorrelation(act1,act2):
    return frechet_covariance(act1,act2).sum()/divariance(act1,act2).sum()


# experimental... not quite right
def group_sigma_product(act1,act2):
    sigma1, sigma2 = cov(act1, rowvar=True), cov(act2, rowvar=True)
    return (sqrtm(sigma1)*sqrtm(sigma2)).sum(axis=-1)
    #return np.sqrt(sigma1.sum(axis=-1))*np.sqrt(sigma2.sum(axis=-1))
# experimental... not quite right
def group_dirrelation(act1, act2):
    return group_sigma_product(act1,act2).sum()/divariance(act1,act2).sum()
# experimental... not quite right
def group_correlation(act1, act2):
    return frechet_covariance(act1,act2).sum()/group_sigma_product(act1,act2).sum()

# dirrelation and divariance are calculated features wise without diffence in process.  Different than the pattern matching of the frechet covariance.  
# I'll have to see if portfolio dirrelation makes sense as is.  maybe not.  prob sigma_product/total divariance
_="""
act1 = random(10*11)
act1 = act1.reshape((10,11))
act2 = random(10*11)
act2 = act2.reshape((10,11))
"""
distr_default = {'mean1':0, 'mean2':2, 'var1':1, 'var2':1, 'corr':0.0, 'size':1000}
act1,act2 = generate_distributions(**distr_default)
act1=act1.reshape(10,100)
act2=act2.reshape(10,100)

sigma1 = cov(act1, rowvar=True)
sigma2 = cov(act2, rowvar=True)

mu1, mu2 = act1.mean(axis=1),act2.mean(axis=1),
ssdiff = np.sum((mu1 - mu2)**2.0)

covmean = sqrtm(sigma1.dot(sigma2)).real
covmean.shape

(10, 10)

In [25]:
#Means of creating a 2 distributions with different frechet covariancde... the pattern between features must be different
holder = []
distr_corr=distr_default.copy()
for c in np.arange(-1,1,0.4):
    distr_corr['corr'] = c
    a,b = generate_distributions(**distr_corr)
    holder.append(a)
    holder.append(b)
act1 = np.array(holder)
#act2 = act1[np.random.choice(np.arange(10),10,replace=False)]


In [26]:
cut = 3
x = list(np.arange(10)[:10-cut])
x.extend(np.random.choice(np.arange(10),cut,replace=True))
act2=act1[x]
act2.shape

(10, 1000)

### Multi-variational distance
Good name.  Based on metrics derived from variance over multiple features.  True distance metric.  
Leads to multi-dicorrelation - scaled version that captures both the magnitude and direcional differences between collections of multi-feature samples.

I just ran some differences in the pattern between the distribution features - to change the frechet covariance.  
When I change 5 of 10 features (scramble them), the effective frechet correlation goes to 0.75.  When I change 4(60% agreement), it goes to 0.85.  
I suspect that the frech_corr will vary by the root of the difference in features (assuming perfect syhmmentry for the remaining features).  
So diff in features would be something akin to R^2... may not be perfect, but interesting observation.

In [27]:
print("fid",fid(act1, act2))
print("vardist",vardist(act1,act2)*2)
print("diveriance",divariance(act1,act2).mean())
print("frechet_cov",frechet_covariance(act1,act2).mean())
print("frechet_dicorr",frechet_dicorrelation(act1,act2))
print("dirrelation",dirrelation(act1,act2).mean())

print(np.sqrt(   divariance(act1,act2).sum()   *   frechet_covariance(act1,act2).sum()   ))
print(divariance(act1,act2).sum())
print(frechet_covariance(act1,act2).sum())
denom_alt = np.sqrt(trace(sigma1)*trace(sigma2))
print("denom_alt",denom_alt)
denom = trace(sigma1*sigma2)
denom_d = np.diag(sigma1*sigma2).sum()
print("denom",denom)
print("denom_d",denom_d)

#none of the denomenators have been quite right.  Its not crucial right now.  we can get to the combined number, which seems to behave properly.
print('dirr alt',denom/divariance(act1,act2).sum())
print('corr alt',frechet_covariance(act1,act2).sum()/denom)

print('dirr alt',denom_alt/divariance(act1,act2).sum())
print('corr alt',frechet_covariance(act1,act2).sum()/denom_alt)

fid 10.94778521518328
vardist 10.931487907669577
diveriance 1.4181226762363957
frechet_cov 0.8715482808529167
frechet_dicorr 0.614578904531693
dirrelation 0.8657648210218378
11.117384496869612
14.181226762363956
8.715482808529167
denom_alt 9.899281068589497
denom 9.794172479189902
denom_d 9.794172479189902
dirr alt 0.6906435277646777
corr alt 0.889864133702702
dirr alt 0.6980553399556052
corr alt 0.8804157340459267


In [636]:
print(group_dirrelation(act1,act2))
print((frechet_covariance(act1,act2)/(act1.var(axis=1)*act2.var(axis=1))).mean())
print(((act1.var(axis=1)*act2.var(axis=1))/divariance(act1,act2)).mean())

print(group_correlation(act1,act2))
frechet_dicorrelation(act1,act2)

(0.8564539818555219+8.920933902317254e-10j)
0.8704520790377721
0.9959676504340763
(1.0100719648026095-1.0521038404265204e-09j)


np.float64(0.8650801562158256)

In [591]:
print(sigma_product(act1,act2),act1.var(axis=1)*act2.var(axis=1))
print(frechet_covariance(act1,act2)/sigma_product(act1,act2))
print(frechet_covariance(act1,act2)/(act1.var(axis=1)*act2.var(axis=1)))
np.corrcoef(act1).sum(axis=0)/2
for i in np.arange(act1.shape[0]):
    print(np.corrcoef(act1[i],act2[i])[0,1])

[0.25732598        nan 0.33379802 0.61607698 0.89803976 1.00154344
        nan 0.20945457 1.37095678 0.9126839 ] [1.03588853 1.07446886 0.88762688 0.92242925 0.9197901  0.95719329
 0.96108874 1.02334861 1.03962114 0.97595371]
[1.59315571        nan 2.56955988 1.48364511 1.05524339 0.96688365
        nan 3.21300623 0.70090177 0.97676508]
[0.3957572  0.42491626 0.96630018 0.99090482 1.03028997 1.0116828
 0.64975812 0.65762424 0.92428481 0.91344267]
-0.026480155867643212
-0.013273951558852195
1.0
0.010283574160508313
1.0
0.025203161693953324
-0.00840248518692902
0.026577282210540547
-0.008845374634954169
-0.010583566183575857


C:\Users\doste\AppData\Local\Temp\ipykernel_16208\904438989.py:51: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(sigma1.sum(axis=-1)*sigma2.sum(axis=-1))


In [474]:
print(divariance(act1,act2).sum())
print(trace(covmean))
print(divariance(act1,act2).sum()-trace(covmean))
print(divariance(act1,act2).sum()-np.sum(covmean))

10.228572578382682
10.228769800635416
-0.0001972222527335532
-3.8959582458604487


In [531]:
divariance(act1,act2).sum()-(sigma1.sum()+sigma2.sum())/2
print(trace(covmean)-trace(sigma1+sigma2)/2)
print(divariance(act1,act2).sum()-(sigma1+sigma2).sum()/2)
print(divariance(act1,act2).sum()-trace(sigma1+sigma2)/2)


-1.1667703896439185
3.2308816817157737
0.07514604816387838


np.float64(1.241916437807797)

In [476]:
print(((sqrtm(sigma1)*sqrtm(sigma2)).sum(axis=1))/divariance(act1,act2))
np.sqrt(np.diag(sigma1)*np.diag(sigma2))/divariance(act1,act2)

[1.00005893 1.00000502 1.00000013 1.         1.00005161 1.00003488
 1.00001317 1.00000457 1.00002362 1.00000331]


array([1.00005893, 1.00000502, 1.00000013, 1.        , 1.00005161,
       1.00003488, 1.00001317, 1.00000457, 1.00002362, 1.00000331])

In [590]:
print(divariance(act1,act2)-np.diag(covmean))
print((2*divariance(act1,act2)-np.diag(sigma1)-np.diag(sigma2))/2)

print((divariance(act1,act2)-np.diag(covmean)).sum())
print(vardist(act1,act2,return_features=False))
divariance(act1,act2).sum()-trace(covmean)

[ 0.03580629  0.07144336  0.01276262 -0.11303699  0.02441699  0.12260161
  0.07845925 -0.03853722  0.03376722 -0.00306455]
[ 0.01595496  0.0568716  -0.00353341 -0.13136284  0.01698707  0.08887527
  0.04287239 -0.05235427  0.00460522 -0.03310641]
0.22461856960443893
2.2068687774486975


np.float64(0.22461856960443782)

In [589]:
print(ssdiff)
print(trace(sigma1 + sigma2 - 2.0 * covmean))
print((ssdiff+trace(sigma1 + sigma2 - 2.0 * covmean))/2)
print(fid(act1, act2))

0.3296056036080203
0.4376180076847569
0.38361180564638864
4.413742769405473


So when you double the difference between divariance.sum() and trace(covmean) then you approximate frechet.  but you don't have to include nearly as many components and don't have to separate mean and variance and add them... although I suppose its validated because they work so well.  Basically, this basic premise of divar vs covar really does capture differencdes in distributions.  If you want this normalized, you can divide them.  If you want a raw number, take the difference.  beautiful.

- divariance.sum() - trace(covmean) = divariance (variational?) inception distance ~ 1/2 fid - for normal distributions with differing mean, variance, correlation
- trace(sqrtm(sigma1*sigma2)) might be equal to or approximate covmean : trace(sqrtm(sigma1.dot(sigma2)).real)... but no reason to substitute
- if you want a normalized group similarity metric instead of a distance: trace(covmean)/divariance.sum().  it will vary between -1 and 1... same concept as dicorrelation.  this accounts for multiple features.  Not sure what to call it - something that signifies dicorrelation and multiple features and distance - portfolio dicorrelation?   .Doesn't assume a basis - dimensions can be correlated.  This could be useful for clustering, although I think a origin established between clusters prob works better.
- Note that this should have application whenever there are multiple samples over multiple features and two distinct categories... that applies to tuning models on the same set of image/text data, classifying based on multiple features of the same model, clustering observations within a single data set/model.
- I should add divariance.sum() - 0.5(trace(sigma1+sigma2) could be used if you don't want to factor in covar... but not sure why not.
- 
what other equilvalence do I need to keep in mind?  or to use to get to that answer.  Not that I'm trying to prove it right now.  Divariance is simply the feature-wise divarance... no adjustment, no weighting.  The trace of covmean is the dot of the cov matrices... tells you how well their interdependencies match up.  Now, I could add divar matrix to the mix... but I think it works out in the wash.  At some point it may make sense to understand the single model internal var... to get a shape of direction and magnitude, but seems like work for the future.

In [459]:
trace(sqrtm(sigma1*sigma2))

np.float64(15.453894336072858)

In [413]:
print(divariance(act1,act2))
np.diag(covmean)
#np.diag(covmean).sum()

[1.52917151 1.46907818 1.48750257 1.40252402 1.76328241 1.72305859
 1.82827657 1.5725433  1.45738332 1.47130109]


array([1.43578388, 1.28890683, 1.36906847, 1.26244638, 1.57981819,
       1.58146596, 1.7007222 , 1.48561484, 1.39469606, 1.26863351])

In [415]:
act1.mean(axis=1)-act2.mean(axis=1)

array([-0.04873507,  0.1478392 , -0.16321923,  0.06052181, -0.40766243,
        0.03043743, -0.03916117,  0.01701147, -0.02863463, -0.16625224])

In [367]:
print((sigma1).sum()/10)
#sigma2.sum()/10

trace(sigma1)/10

1.0284085546710189


np.float64(0.9603358488965279)

In [368]:
print(divariance(act1,act2).mean())
print(np.var(act1))
print(np.var(act2))

0.9504941526493063
0.9615990286432761
0.9615990286432757


In [188]:
distr_default = {'mean1':0, 'mean2':0, 'var1':1, 'var2':1, 'corr':1, 'size':110}
act1,act2 = generate_distributions(**distr_default)
act1=act1.reshape(10,11)
act2=act2.reshape(10,11)

(10, 11)